**Table of contents**<a id='toc0_'></a>    
- [Import packages](#toc1_)    
- [Set up](#toc2_)    
  - [Define paths](#toc2_1_)    
- [Load raw footfall data](#toc3_)    
- [Raw data stats](#toc4_)    
  - [summary](#toc4_1_)    
  - [check duplicates](#toc4_2_)    
- [Process duplicated sensors in sensor location data](#toc5_)    
- [Split historical footfall data by year](#toc6_)    
- [Merge historical footfall data, current footfall data and sensor location data](#toc7_)    
  - [status](#toc7_1_)    
  - [double check](#toc7_2_)    
  - [Split data based on covid period](#toc7_3_)    
  - [Split data based on year](#toc7_4_)    
- [Handling missing values (without interpolation)](#toc8_)    
  - [Extract year span for each sensor](#toc8_1_)    
    - [Save data](#toc8_1_1_)    
  - [Calculate missing rate and plot data](#toc8_2_)    
  - [Remove sensor with missing data rate >= 50%](#toc8_3_)    
  - [Create segments](#toc8_4_)    
- [Process features (TBD)](#toc9_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
# !pip install -q tsfresh stumpy mplcursors plotly kaleido # colab
# # !pip install -q tsfresh==0.20.1 stumpy==1.12.0 missingno==0.5.2 geopy==2.4.0 mplcursors==0.5.2 # local machine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.3/95.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.0/89.0 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.4 MB/s eta 0:00:00


# <a id='toc1_'></a>[Import packages](#toc0_)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import osmnx as ox
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import plotly.express as px
import missingno as msno
import seaborn as sns
from pathlib import Path
from datetime import timedelta, datetime
from geopy.distance import geodesic
from tsfresh import extract_features
from tsfresh.feature_selection.selection import select_features
from tsfresh.utilities.dataframe_functions import impute
from math import sin, cos, sqrt, atan2, radians
from tabulate import tabulate
from importlib import reload

import sys
if Path('/content/drive/MyDrive').exists():
  sys.path.append('/content/drive/MyDrive/Colab Notebooks/custom_modules')
else:
  sys.path.append('./custom_modules')

import basic_funs, plot_funs
reload(basic_funs)
reload(plot_funs)
from basic_funs import *
from plot_funs import *

# <a id='toc2_'></a>[Set up](#toc0_)

In [31]:
save_subdirs = {
  "original": "0. original_data",
  "merged": "1. merged_peds_data_hist_curr",
  "missing_rate": "2. missing_rate",
  "remove_sensors": "3. remove_sensors_with_high_missing_rate",
  "final_group": "4. final_group",
  "add_fea": "5. join_features",
}

save_subdirs = Config(**save_subdirs)

rewrite = True

## <a id='toc2_1_'></a>[Define paths](#toc0_)

In [3]:
footfall_counts_path_min = 'pedestrian-counting-system-past-hour-counts-per-minute'
footfall_counts_path1 = 'pedestrian-counting-system-monthly-counts-per-hour'
footfall_counts_path2 = 'Pedestrian_Counting_System_Monthly_counts_per_hour_may_2009_to_14_dec_2022'
sensor_locations_path = 'pedestrian-counting-system-sensor-locations'

local_path = Path('../Data (20230918)')
drive_path = Path('/content/drive/MyDrive/Data/Melbourne_Footfalls')

base_path = local_path if local_path.exists() else drive_path

In [4]:
save_dir = Path('../data_preprocessed') if local_path.exists() else Path('/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls')
if save_dir.exists() == False:
  save_dir.mkdir(parents=True, exist_ok=True)

# <a id='toc3_'></a>[Load raw footfall data](#toc0_)

[Pedestrian Counting System (counts per hour)](https://melbournetestbed.opendatasoft.com/explore/dataset/pedestrian-counting-system-monthly-counts-per-hour/information/)

[Pedestrian Counting System - Sensor Locations](https://melbournetestbed.opendatasoft.com/explore/dataset/pedestrian-counting-system-sensor-locations/information/)

the new data (collected since 2023) has an ISO 8601 datetime format with a timezone offset

while the historical data hasn't timezone-awareness

In [ ]:
footfall_counts_23_today = pd.read_excel(base_path.joinpath(footfall_counts_path1 + '.xlsx'))
footfall_counts_min = pd.read_excel(base_path.joinpath(footfall_counts_path_min + '.xlsx'))
footfall_counts_09_22 = pd.read_csv(base_path.joinpath(footfall_counts_path2 + '.csv'))
sensor_locations = pd.read_excel(base_path.joinpath(sensor_locations_path + '.xlsx'))

# <a id='toc4_'></a>[Raw data stats](#toc0_)

## <a id='toc4_1_'></a>[summary](#toc0_)

In [ ]:
data_stats = {
  "Data Description": [
    "footfall_counts (2009 - 2022)",
    "footfall_counts (2023 - today)",
    "footfall_counts (past hour per min)",
    "sensor_locations"
  ],
  "Size": [
    footfall_counts_09_22.shape,
    footfall_counts_23_today.shape,
    footfall_counts_min.shape,
    sensor_locations.shape
  ],
  "Attributes": [
    footfall_counts_09_22.columns.tolist(),
    footfall_counts_23_today.columns.tolist(),
    footfall_counts_min.columns.tolist(),
    sensor_locations.columns.tolist()
  ],
  "No_Sensor_ID_or_Location_ID": [
    len(footfall_counts_09_22.Sensor_ID.unique()),
    len(footfall_counts_23_today.LocationID.unique()),
    len(footfall_counts_min.Location_ID.unique()),
    len(sensor_locations.Location_ID.unique()),
  ],
  "No_Sensor_Name": [
    len(footfall_counts_09_22.Sensor_Name.unique()),
    np.nan,
    np.nan,
    len(sensor_locations.Sensor_Name.unique()),
  ],
  "No_Sensor_Description": [
    np.nan,
    np.nan,
    np.nan,
    len(sensor_locations.Sensor_Description.unique()),
  ]
}

df_summary = pd.DataFrame(data_stats)
save_data(df_summary, save_dir, save_subdirs.original, 'data_summary.xlsx', rewrite=rewrite)
df_summary

/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/0. original_data/data_summary.xlsx updated.


,Data Description,Size,Attributes,No_Sensor_ID_or_Location_ID,No_Sensor_Name,No_Sensor_Description
0,footfall_counts (2009 - 2022),"(4562230, 10)","[ID, Date_Time, Year, Month, Mdate, Day, Time,...",82,94.0,NaN
1,footfall_counts (2023 - today),"(272259, 5)","[SensingDateTime(Hour), LocationID, Direction_...",81,NaN,NaN
2,footfall_counts (past hour per min),"(32636, 7)","[Location_ID, Sensing_DateTime, Sensing_Date, ...",76,NaN,NaN
3,sensor_locations,"(113, 12)","[Location_ID, Sensor_Description, Sensor_Name,...",113,111.0,110.0


## <a id='toc4_2_'></a>[check duplicates](#toc0_)

especially for data of sensor location

duplicate location of sensor location data:

- Harbour Esplanade (Location_ID: 111, 120 with same geo location)

- Birrarung Marr (Location_ID: 7, 122 with same geo location)

- Birrarung Marr East - Batman Ave Bridge Entry (Location_ID: 123, 124 with different geo location)


In [ ]:
duplicates_23_today = check_duplicates(footfall_counts_23_today, print_msg='footfall_counts_23_today')
duplicates_min = check_duplicates(footfall_counts_min, print_msg='footfall_counts_min')
duplicates_09_22 = check_duplicates(footfall_counts_09_22, print_msg='footfall_counts_09_22')

# duplicates in sensor location data
columns_to_check = [
  'all',
  ['Sensor_Description'],
  ['Sensor_Name'],
  ['Sensor_Name', 'Location'],
  ['Sensor_Description', 'Location'],
  ['Location'],
  ['Location_ID']
]

duplicate_dfs = [check_duplicates(sensor_locations, col, print_msg='sensor_locations') for col in columns_to_check]

duplicates_sensor_locations = pd.concat(duplicate_dfs, axis=0)
duplicates_sensor_locations.drop_duplicates(inplace=True)
duplicates_sensor_locations.dropna(axis=1, how='all', inplace=True)
save_data(duplicates_sensor_locations, save_dir, save_subdirs.original, 'duplicates_sensor_locations.xlsx', index=True, rewrite=rewrite)

Duplicates in footfall_counts_23_today: 0
Duplicates in footfall_counts_min: 0
Duplicates in footfall_counts_09_22: 0
Duplicates in sensor_locations: 0
Duplicates in sensor_locations ['Sensor_Description']: 6
+-----+---------------+-----------------------------------------------+---------------+---------------------+--------+-----------------+----------+---------------+---------------+------------+-------------+----------------------------+
|     |   Location_ID | Sensor_Description                            | Sensor_Name   | Installation_Date   | Note   | Location_Type   | Status   | Direction_1   | Direction_2   |   Latitude |   Longitude | Location                   |
+=====+===============+===============================================+===============+=====================+========+=================+==========+===============+===============+============+=============+============================+
|  28 |           120 | Harbour Esplanade                             | HarbBlix_T 

# <a id='toc5_'></a>[Process duplicated sensors in sensor location data](#toc0_)

the current data only has LocationID without sensor description and sensor name

In [ ]:
# duplicates_sensor_locations.reset_index(drop=True, inplace=True)
duplicates_sensor_locations

,Location_ID,Sensor_Description,Sensor_Name,Installation_Date,Note,Location_Type,Status,Direction_1,Direction_2,Latitude,Longitude,Location
28,120,Harbour Esplanade,HarbBlix_T,2021-03-25,NaN,Outdoor Blix,A,NaN,NaN,-37.814426,144.944366,"-37.81442577, 144.94436558"
63,111,Harbour Esplanade,HarbBlix_T,2021-03-25,NaN,Outdoor Blix,A,NaN,NaN,-37.814426,144.944366,"-37.81442577, 144.94436558"
66,122,Birrarung Marr,FedCycle_T,2022-08-23,NaN,Outdoor,A,East,West,-37.818629,144.971694,"-37.81862929, 144.97169395"
67,124,Birrarung Marr East - Batman Ave Bridge Entry,BirBridge_T,2023-06-02,NaN,Outdoor,A,North,South,-37.817574,144.973299,"-37.81757416, 144.97329934"
72,7,Birrarung Marr,FedPed_T,2014-12-17,NaN,Outdoor,A,East,West,-37.818629,144.971694,"-37.81862929, 144.97169395"
111,123,Birrarung Marr East - Batman Ave Bridge Entry,BirBridge_T,2023-06-02,<Null>,Outdoor,A,East,West,-37.817537,144.973297,"-37.81753741, 144.97329734"


In [ ]:
# check if these id in historical data and current data
print(footfall_counts_09_22[footfall_counts_09_22['Sensor_ID'].isin(duplicates_sensor_locations['Location_ID'])]['Sensor_ID'].unique())
print(footfall_counts_23_today[footfall_counts_23_today['LocationID'].isin(duplicates_sensor_locations['Location_ID'])]['LocationID'].unique())

[7]
[122   7 123 124]


In [ ]:
# find sensors with same geo location but different Location_ID / Sensor_Description / Sensor_Name
duplicate_sensors = duplicates_sensor_locations[duplicates_sensor_locations[['Location']].duplicated(keep=False)]
duplicate_sensors

,Location_ID,Sensor_Description,Sensor_Name,Installation_Date,Note,Location_Type,Status,Direction_1,Direction_2,Latitude,Longitude,Location
28,120,Harbour Esplanade,HarbBlix_T,2021-03-25,NaN,Outdoor Blix,A,NaN,NaN,-37.814426,144.944366,"-37.81442577, 144.94436558"
63,111,Harbour Esplanade,HarbBlix_T,2021-03-25,NaN,Outdoor Blix,A,NaN,NaN,-37.814426,144.944366,"-37.81442577, 144.94436558"
66,122,Birrarung Marr,FedCycle_T,2022-08-23,NaN,Outdoor,A,East,West,-37.818629,144.971694,"-37.81862929, 144.97169395"
72,7,Birrarung Marr,FedPed_T,2014-12-17,NaN,Outdoor,A,East,West,-37.818629,144.971694,"-37.81862929, 144.97169395"


In [ ]:
duplicates_sensor_locations[~duplicates_sensor_locations[['Location']].duplicated(keep=False)]

,Location_ID,Sensor_Description,Sensor_Name,Installation_Date,Note,Location_Type,Status,Direction_1,Direction_2,Latitude,Longitude,Location
67,124,Birrarung Marr East - Batman Ave Bridge Entry,BirBridge_T,2023-06-02,NaN,Outdoor,A,North,South,-37.817574,144.973299,"-37.81757416, 144.97329934"
111,123,Birrarung Marr East - Batman Ave Bridge Entry,BirBridge_T,2023-06-02,<Null>,Outdoor,A,East,West,-37.817537,144.973297,"-37.81753741, 144.97329734"


If the Sensor_Description is the same, but the Sensor_Name and Location_ID are different, the names and IDs should be aggregated together in the New_Sensor_Name column.

For the same Sensor_Description, the Sensor_Name and Location_ID pairs should be ordered by Location_ID.

In [ ]:
processed_sensors = process_sensor_loc_data(sensor_locations)
save_data(processed_sensors, save_dir, save_subdirs.merged, 'sensor_locations_processed.xlsx')

The initial number of locations is 113
The number of unique New_Sensor_Names after merging is 111
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/1. merged_peds_data_hist_curr/sensor_locations_processed.xlsx updated.


then double check if the process correctly

In [ ]:
processed_sensors[processed_sensors['Sensor_Name'] == 'HarbBlix_T']

,Location_ID,Sensor_Description,Sensor_Name,New_Sensor_Name,Installation_Date,Note,Location_Type,Status,Direction_1,Direction_2,Latitude,Longitude,Location
28,120,Harbour Esplanade,HarbBlix_T,"Harbour Esplanade | HarbBlix_T [111, 120]",2021-03-25,NaN,Outdoor Blix,A,NaN,NaN,-37.814426,144.944366,"-37.81442577, 144.94436558"
63,111,Harbour Esplanade,HarbBlix_T,"Harbour Esplanade | HarbBlix_T [111, 120]",2021-03-25,NaN,Outdoor Blix,A,NaN,NaN,-37.814426,144.944366,"-37.81442577, 144.94436558"


In [ ]:
processed_sensors[processed_sensors['Sensor_Description'] == 'Birrarung Marr']

,Location_ID,Sensor_Description,Sensor_Name,New_Sensor_Name,Installation_Date,Note,Location_Type,Status,Direction_1,Direction_2,Latitude,Longitude,Location
66,122,Birrarung Marr,FedCycle_T,"Birrarung Marr | FedPed_T, FedCycle_T [7, 122]",2022-08-23,NaN,Outdoor,A,East,West,-37.818629,144.971694,"-37.81862929, 144.97169395"
72,7,Birrarung Marr,FedPed_T,"Birrarung Marr | FedPed_T, FedCycle_T [7, 122]",2014-12-17,NaN,Outdoor,A,East,West,-37.818629,144.971694,"-37.81862929, 144.97169395"


In [ ]:
processed_sensors[processed_sensors['Sensor_Name'] == 'BirBridge_T']

,Location_ID,Sensor_Description,Sensor_Name,New_Sensor_Name,Installation_Date,Note,Location_Type,Status,Direction_1,Direction_2,Latitude,Longitude,Location
67,124,Birrarung Marr East-Batman Ave Bridge Entry,BirBridge_T,Birrarung Marr East-Batman Ave Bridge Entry | ...,2023-06-02,NaN,Outdoor,A,North,South,-37.817574,144.973299,"-37.81757416, 144.97329934"
111,123,Birrarung Marr East-Batman Ave Bridge Entry,BirBridge_T,Birrarung Marr East-Batman Ave Bridge Entry | ...,2023-06-02,<Null>,Outdoor,A,East,West,-37.817537,144.973297,"-37.81753741, 144.97329734"


In [ ]:
processed_sensors[processed_sensors['Sensor_Name'] == 'BirBridge_T']['New_Sensor_Name'].unique()

array(['Birrarung Marr East-Batman Ave Bridge Entry | BirBridge_T [124]',
       'Birrarung Marr East-Batman Ave Bridge Entry | BirBridge_T [123]'],
      dtype=object)

# <a id='toc6_'></a>[Split historical footfall data by year](#toc0_)

In [ ]:
for year, group in footfall_counts_09_22.groupby('Year'):
  save_data(group, save_dir, save_subdirs.original, f"data_for_year_{year}.xlsx", rewrite=rewrite)

footfall_counts_23_today['SensingDateTime(Hour)'] = pd.to_datetime(footfall_counts_23_today['SensingDateTime(Hour)'], utc=True)
footfall_counts_23_today['SensingDateTime(Hour)'] = footfall_counts_23_today['SensingDateTime(Hour)'].dt.tz_convert('Australia/Sydney')
footfall_counts_23_today['SensingDateTime(Hour)'] = footfall_counts_23_today['SensingDateTime(Hour)'].dt.tz_localize(None)

save_data(footfall_counts_23_today, save_dir, save_subdirs.original, "data_for_year_2023.xlsx", rewrite=rewrite)

/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/0. original_data/data_for_year_2009.xlsx updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/0. original_data/data_for_year_2010.xlsx updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/0. original_data/data_for_year_2011.xlsx updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/0. original_data/data_for_year_2012.xlsx updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/0. original_data/data_for_year_2013.xlsx updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/0. original_data/data_for_year_2014.xlsx updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/0. original_data/data_for_year_2015.xlsx updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/0. original_data/data_for_year_2016.xlsx updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/0. original_data/data_for_year_2017.xlsx updated.
/content/drive/MyDrive/ProcessedData_

# <a id='toc7_'></a>[Merge historical footfall data, current footfall data and sensor location data](#toc0_)

footfall_counts_09_22, footfall_counts_23_today and sensor_locations have different column names, so have to think a way to unify the column names for further processing
- footfall_counts_09_22: Sensor_ID, Sensor_Name
- footfall_counts_23_today: LocationID
- sensor_locations: Location_ID, Sensor_Description, Sensor_Name

(the Sensor_Description of sensor_locations seems similar to Sensor_Name of footfall_counts_23_today)

['Sensor_Name', 'Sensor_ID'] in footfall_counts_09_22 is equivalent to ['Sensor_Description', 'Location_ID'] in sensor_locations

In [ ]:
hist_df, curr_df = format_datetime_remove_duplicates(footfall_counts_09_22, footfall_counts_23_today)
save_data(hist_df, save_dir, save_subdirs.merged, 'footfall_counts_09_22.csv', rewrite=rewrite)
save_data(curr_df, save_dir, save_subdirs.merged, 'footfall_counts_23_today.xlsx', rewrite=rewrite)

manually remove the timezone
Timezone information has been successfully removed!
Number of duplicate rows in historical data: 0
Number of duplicate rows in current data: 0
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/1. merged_peds_data_hist_curr/footfall_counts_09_22.csv updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/1. merged_peds_data_hist_curr/footfall_counts_23_today.xlsx updated.


In [ ]:
# The pair of ['Sensor_Name', 'Sensor_ID'] in hist_df should equivalent to
# ['Sensor_Description', 'Location_ID'] in sensor data.
final_merged_df, uncommon_pairs = merge_hist_curr_sensor_data(hist_df, curr_df, processed_sensors)

1. Find if ['Sensor_Name', 'Sensor_ID'] in footfall_counts_09_22 is equivalent to ['Sensor_Description', 'Location_ID'] in sensor_locations
The similarity is 81.56%
The pair of ['Sensor_Name', 'Sensor_ID'] in data1 may equivalent to ['Sensor_Description', 'Location_ID'] in data2.
2. Find if ['Sensor_ID', 'Sensor_Name'] in footfall_counts_09_22 is equivalent to ['Location_ID', 'Sensor_Name'] in sensor_locations
The similarity is 0.00%
They are NOT equivalent
['Sensor_Name', 'Sensor_ID'] in footfall_counts_09_22 is equivalent to ['Sensor_Description', 'Location_ID'] in sensor_locations
The shape of final_merged_df before grouping 4400587
The shape of final_merged_df after grouping 4378809


In [ ]:
save_data(final_merged_df, save_dir, save_subdirs.merged, "footfall_merged.csv", rewrite=rewrite)

/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/1. merged_peds_data_hist_curr/footfall_merged.csv updated.


## <a id='toc7_1_'></a>[status](#toc0_)

In [ ]:
uncommon_pairs

,Sensor_ID,Sensor_Name,Location_ID,Sensor_Description
635166,46,Pelham St (S),46,Pelham St (South)
942243,54,Lincoln-Swanston(West),54,Lincoln-Swanston (West)
999604,56,Lonsdale St - Elizabeth St (North),56,Lonsdale St-Elizabeth St (North)
1070358,58,Bourke St - Spencer St (North),58,Bourke St-Spencer St (North)
1106166,59,Swanston St - RMIT Building 80,59,Building 80 RMIT
1138638,61,Swanston St - RMIT Building 14,61,RMIT Building 14
1525165,53,Collins St (North),53,Collins Street (North)
3878069,64,Royal Pde - Grattan St,64,Royal Pde-Grattan St
3882509,65,Swanston St - City Square,65,Swanston St-City Square
3919421,66,State Library - New,66,State Library-New


In [ ]:
# find the rows where the values in the LocationID column of footfall_counts_23_today are NOT found
# in the Location_ID column of the sensor_locations DataFrame.
footfall_counts_23_today[~footfall_counts_23_today['LocationID'].isin(sensor_locations['Location_ID'])]

,SensingDateTime(Hour),LocationID,Direction_1,Direction_2,Total_of_Directions


## <a id='toc7_2_'></a>[double check](#toc0_)

In [ ]:
unique_sensor_ids1 = footfall_counts_09_22[~footfall_counts_09_22['Sensor_ID'].isin(sensor_locations['Location_ID'])]['Sensor_ID'].unique()
unique_sensor_ids2 = footfall_counts_23_today[~footfall_counts_23_today['LocationID'].isin(sensor_locations['Location_ID'])]['LocationID'].unique()
unique_sensor_ids = list(unique_sensor_ids1) + list(unique_sensor_ids2)
removed = footfall_counts_09_22['Sensor_ID'].isin(unique_sensor_ids).sum() + footfall_counts_23_today['LocationID'].isin(unique_sensor_ids).sum()
print(f"These sensors may have been removed: {unique_sensor_ids}")
print(f"{removed} data may be removed.")

These sensors may have been removed: [34, 15, 22, 13, 16, 32, 33, 38, 60]
433359 data may be removed.


In [ ]:
final_merged_df.shape

(4378809, 14)

In [ ]:
footfall_counts_09_22.shape[0] + footfall_counts_23_today.shape[0] - final_merged_df.shape[0]

455680

In [ ]:
final_merged_df.head()

,Date_Time,New_Sensor_Name,Hourly_Counts,Location_ID,Installation_Date,Location_Type,Status,Latitude,Longitude,Location,Year,Month,MDate,Day
0,2009-05-01,Bourke Street Mall (North) | Bou292_T [1],53,1,2009-03-24,Outdoor,A,-37.813494,144.965153,"-37.81349441, 144.96515323",2009,5,1,Friday
1,2009-05-01,Bourke Street Mall (South) | Bou283_T [2],52,2,2009-03-30,Outdoor,A,-37.813807,144.965167,"-37.81380668, 144.96516718",2009,5,1,Friday
2,2009-05-01,Collins Place (North) | Col12_T [18],36,18,2009-03-30,Outdoor,A,-37.813449,144.973054,"-37.81344862, 144.97305353",2009,5,1,Friday
3,2009-05-01,Collins Place (South) | Col15_T [17],28,17,2009-03-30,Outdoor,A,-37.813625,144.973236,"-37.81362543, 144.97323591",2009,5,1,Friday
4,2009-05-01,Flinders Street Station Underpass | FliS_T [6],139,6,2009-03-25,Outdoor,A,-37.819117,144.965583,"-37.81911705, 144.96558255",2009,5,1,Friday


In [ ]:
# check sensor_name and Location
len(sorted(final_merged_df['New_Sensor_Name'].unique()))

80

In [ ]:
len(final_merged_df['Location'].unique())

80

In [ ]:
final_merged_df[final_merged_df['Location_ID'].isin([120, 111, 122, 124, 7, 123])][['New_Sensor_Name', 'Location']].drop_duplicates()

,New_Sensor_Name,Location
48480,"Birrarung Marr | FedPed_T, FedCycle_T [7, 122]","-37.81862929, 144.97169395"
4329483,Birrarung Marr East-Batman Ave Bridge Entry | ...,"-37.81753741, 144.97329734"
4329484,Birrarung Marr East-Batman Ave Bridge Entry | ...,"-37.81757416, 144.97329934"


## <a id='toc7_3_'></a>[Split data based on covid period](#toc0_)

In [ ]:
split_and_save_data(final_merged_df, save_dir, save_subdirs.merged, rewrite=rewrite)

## <a id='toc7_4_'></a>[Split data based on year](#toc0_)

In [ ]:
for year, group in final_merged_df.groupby('Year'):
  save_data(group, save_dir, save_subdirs.merged, f"footfall_merged_for_year_{year}.xlsx", rewrite=rewrite)

/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/1. merged_peds_data_hist_curr/footfall_merged_for_year_2009.xlsx updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/1. merged_peds_data_hist_curr/footfall_merged_for_year_2010.xlsx updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/1. merged_peds_data_hist_curr/footfall_merged_for_year_2011.xlsx updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/1. merged_peds_data_hist_curr/footfall_merged_for_year_2012.xlsx updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/1. merged_peds_data_hist_curr/footfall_merged_for_year_2013.xlsx updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/1. merged_peds_data_hist_curr/footfall_merged_for_year_2014.xlsx updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/1. merged_peds_data_hist_curr/footfall_merged_for_year_2015.xlsx updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/1. merged_peds_data_hist_c

# <a id='toc8_'></a>[Handling missing values (without interpolation)](#toc0_)
the data generated from this section is used for online learning

## <a id='toc8_1_'></a>[Extract year span for each sensor](#toc0_)

In [ ]:
save_path = save_dir / save_subdirs.missing_rate
save_path.mkdir(parents=True, exist_ok=True)
save_path

PosixPath('/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/2. missing_rate')

In [ ]:
df = pd.read_csv(save_dir /save_subdirs.merged / "footfall_merged.csv")

# Ensure Date_Time is a pandas datetime object
df['Date_Time'] = pd.to_datetime(df['Date_Time'])

# Group by sensor name and find time span for each sensor
# time_spans = df.groupby('Sensor_Name').agg({'Date_Time': ['min', 'max']})
# year_spans = df.groupby('Sensor_Name').agg({'Date_Time': [lambda x: x.dt.year.min(), lambda x: x.dt.year.max()]})
time_spans = df.groupby('New_Sensor_Name').agg({'Date_Time': ['min', 'max']})
year_spans = df.groupby('New_Sensor_Name').agg({'Date_Time': [lambda x: x.dt.year.min(), lambda x: x.dt.year.max()]})
year_spans.columns = ['Start_Year', 'End_Year']

unique_start_years = sorted(year_spans['Start_Year'].unique())

data_slices = {}

# For each unique starting year, grab data for the sensors active in that period
for i, start_year in enumerate(unique_start_years):
  # sensors starting before or during the current start year
  sensors_to_include = year_spans[year_spans['Start_Year'] <= start_year].index.tolist()

  if i == len(unique_start_years) - 1:
    end_year = df['Date_Time'].dt.year.max()  # Last year of the dataset
  else:
    end_year = unique_start_years[i + 1] - 1  # A year before the next starting year

  # Extract data
  # data_slice = df[(df['Sensor_Name'].isin(sensors_to_include)) &
  #                 (df['Date_Time'].dt.year >= start_year) &
  #                 (df['Date_Time'].dt.year <= end_year)]
  data_slice = df[(df['New_Sensor_Name'].isin(sensors_to_include)) &
                (df['Date_Time'].dt.year >= start_year) &
                (df['Date_Time'].dt.year <= end_year)]

  data_slices[(start_year, end_year)] = data_slice

In [ ]:
# Define a dictionary to store time span and corresponding sensors
time_span_and_sensors = {}

for (start_year, end_year), data_slice in data_slices.items():
  # sensors_in_slice = data_slice['Sensor_Name'].unique()
  sensors_in_slice = data_slice['New_Sensor_Name'].unique()
  time_span_and_sensors[(start_year, end_year)] = sensors_in_slice

print(f"The total time spans: {len(time_span_and_sensors)}")

for time_span, sensors in time_span_and_sensors.items():
  print(f"Time Span: {time_span[0]} - {time_span[1]} ({len(sensors)} sensors)")
  # print("Sensors:")
  # for sensor in sensors:
  #     print(f"  {sensor}")
  # print()

The total time spans: 12
Time Span: 2009 - 2012 (15 sensors)
Time Span: 2013 - 2013 (27 sensors)
Time Span: 2014 - 2014 (28 sensors)
Time Span: 2015 - 2015 (35 sensors)
Time Span: 2016 - 2016 (36 sensors)
Time Span: 2017 - 2017 (45 sensors)
Time Span: 2018 - 2018 (50 sensors)
Time Span: 2019 - 2019 (53 sensors)
Time Span: 2020 - 2020 (63 sensors)
Time Span: 2021 - 2021 (69 sensors)
Time Span: 2022 - 2022 (73 sensors)
Time Span: 2023 - 2023 (80 sensors)


In [ ]:
# original data
data = df.pivot(index='New_Sensor_Name', columns='Date_Time', values='Hourly_Counts')
data.shape

(80, 121501)

In [ ]:
data.head()

Date_Time,2009-05-01 00:00:00,2009-05-01 01:00:00,2009-05-01 02:00:00,2009-05-01 03:00:00,2009-05-01 04:00:00,2009-05-01 05:00:00,2009-05-01 06:00:00,2009-05-01 07:00:00,2009-05-01 08:00:00,2009-05-01 09:00:00,...,2023-09-18 16:00:00,2023-09-18 17:00:00,2023-09-18 18:00:00,2023-09-18 19:00:00,2023-09-18 20:00:00,2023-09-18 21:00:00,2023-09-18 22:00:00,2023-09-18 23:00:00,2023-09-19 00:00:00,2023-09-19 01:00:00
New_Sensor_Name,,,,,,,,,,,,,,,,,,,,,
114 Flinders Street Car Park Crossing | Fli114C_T [118],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.0,61.0,38.0,6.0,8.0,6.0,10.0,3.0,NaN,NaN
114 Flinders Street Car Park Footpath | Fli114F_T [117],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,314.0,423.0,274.0,145.0,123.0,101.0,92.0,32.0,10.0,2.0
231 Bourke St | Bou231_T [63],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,555.0,573.0,516.0,325.0,303.0,235.0,162.0,76.0,42.0,3.0
Alfred Place | AlfPl_T [39],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,140.0,255.0,117.0,47.0,33.0,25.0,22.0,6.0,1.0,NaN
Birrarung Marr East-Batman Ave Bridge Entry | BirBridge_T [123],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,46.0,80.0,66.0,4.0,2.0,2.0,NaN,2.0,NaN,NaN


In [ ]:
plot_time_series_data_iterative(df, save_path)

/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/2. missing_rate updated.


### <a id='toc8_1_1_'></a>[Save data](#toc0_)

In [ ]:
save_data(data, save_dir, save_subdirs.missing_rate, 'footfall_merged.csv', rewrite=rewrite)
save_data(time_spans, save_dir, save_subdirs.missing_rate, 'time_spans.xlsx', rewrite=rewrite)
save_data(year_spans, save_dir, save_subdirs.missing_rate, 'year_spans.xlsx', rewrite=rewrite)

# original data slices
for (start_year, end_year), slice_data in data_slices.items():
  save_data(slice_data, save_dir, save_subdirs.missing_rate, f"original_data_{start_year}_{end_year}.xlsx", rewrite=rewrite)

## <a id='toc8_2_'></a>[Calculate missing rate and plot data](#toc0_)

In [ ]:
missing_rates = {}

for (start_year, end_year), slice_data in data_slices.items():
  print(f"Processing slice {start_year} - {end_year}")
  grouped_slice_data = slice_data.pivot(index='New_Sensor_Name', columns='Date_Time', values='Hourly_Counts')
  save_data(grouped_slice_data, save_dir, save_subdirs.missing_rate, f"grouped_original_{start_year}_{end_year}.csv", rewrite=rewrite)

  # var_name = f"data_slice_{start_year}_{end_year}"
  # globals()[var_name] = slice_data

  missing_rate_per_sensor = grouped_slice_data.isnull().mean(axis=1).sort_values(ascending=False)
  print("Calculated missing rates")
  plot_missing_rate(missing_rate_per_sensor, start_year, end_year, save_path, rewrite=rewrite)
  print(f"Saved missing rate plot for slice {start_year} - {end_year}")

  plot_time_series_data(grouped_slice_data, start_year, end_year, save_path, rewrite=rewrite)
  print(f"Saved time series plot for slice {start_year} - {end_year}")
  plot_time_series_data_sensor(grouped_slice_data, start_year, end_year, save_path, rewrite=rewrite)
  plot_time_series_data_iterative(slice_data, save_path, start_year, end_year, True, rewrite=rewrite)
  print(f"Saved time series plot for each sensor for slice {start_year} - {end_year}")

Processing slice 2022 - 2022
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/2. missing_rate/grouped_original_2022_2022.csv saved.
Calculated missing rates
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/2. missing_rate/missing_values_per_sensor_2022_2022.png updated.
Saved missing rate plot for slice 2022 - 2022
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/2. missing_rate/time_series_data_2022_2022.png updated.
Saved time series plot for slice 2022 - 2022
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/2. missing_rate updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/2. missing_rate updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/2. missing_rate exists and will not be updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/2. missing_rate updated.
Saved time series plot for each sensor for slice 2022 - 2022
Processing slice 2023 - 2023
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/2. missing_

## <a id='toc8_3_'></a>[Remove sensor with missing data rate >= 50%](#toc0_)

In [ ]:
group_processed_slices = []
processed_slices = []

for (start_year, end_year), slice_data in data_slices.items():
  print(f"Processing slice {start_year} - {end_year}")
  grouped_slice_data = slice_data.pivot(index='New_Sensor_Name', columns='Date_Time', values='Hourly_Counts')
  missing_rate_per_sensor = grouped_slice_data.isnull().mean(axis=1).sort_values(ascending=False)

  remove_sensors = missing_rate_per_sensor[missing_rate_per_sensor >= 0.5].index.values

  print(f"Data size before removing: {grouped_slice_data.shape}, {len(remove_sensors)} sensors will be removed.")
  grouped_slice_data.drop(remove_sensors, inplace=True)
  print(f"Data size after removing: {grouped_slice_data.shape}")

  save_data(grouped_slice_data, save_dir, save_subdirs.remove_sensors, f"grouped_original_{start_year}_{end_year}.csv", index=True, rewrite=rewrite)
  group_processed_slices.append(grouped_slice_data)

grouped_combined_data = pd.concat(group_processed_slices, axis=1)
print(f"Size of grouped_combined_data: {grouped_combined_data.shape}")

save_data(grouped_combined_data, save_dir, save_subdirs.remove_sensors, "grouped_combined_data.csv", index=True, rewrite=rewrite)

Processing slice 2009 - 2012
Data size before removing: (15, 31494), 0 sensors will be removed.
Data size after removing: (15, 31494)
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/3. remove_sensors_with_high_missing_rate/grouped_original_2009_2012.csv updated.
Processing slice 2013 - 2013
Data size before removing: (27, 8759), 12 sensors will be removed.
Data size after removing: (15, 8759)
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/3. remove_sensors_with_high_missing_rate/grouped_original_2013_2013.csv updated.
Processing slice 2014 - 2014
Data size before removing: (28, 8759), 2 sensors will be removed.
Data size after removing: (26, 8759)
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/3. remove_sensors_with_high_missing_rate/grouped_original_2014_2014.csv updated.
Processing slice 2015 - 2015
Data size before removing: (35, 8759), 1 sensors will be removed.
Data size after removing: (34, 8759)
/content/drive/MyDrive/ProcessedData_Melbourne_Footfall

## <a id='toc8_4_'></a>[Create segments](#toc0_)
The sensors in each segment have same year span.

In [ ]:
df = pd.read_csv(save_dir / save_subdirs.remove_sensors / 'grouped_combined_data.csv')

df_melted = pd.melt(df, id_vars=['New_Sensor_Name'], var_name='Date_Time', value_name='Hourly_Counts')
df_melted['Date_Time'] = pd.to_datetime(df_melted['Date_Time'])
df_melted['Year'] = df_melted['Date_Time'].dt.year
df_melted = df_melted.dropna(subset=['Hourly_Counts'])

df = df_melted.copy()

# Create a mapping of sensor names to unique integer values
sensors = df['New_Sensor_Name'].unique()
sensor_mapping = {sensor: i for i, sensor in enumerate(sensors)}

# For each year, find which sensors are active
active_sensors_per_year = df.groupby('Year')['New_Sensor_Name'].unique()

# Create segments based on changes in active sensors set
segments = [(active_sensors_per_year.index[0], active_sensors_per_year.index[0])]
for year in active_sensors_per_year.index[1:]:
  # checks if the set of active sensors for the current year is the same as
  # the set of active sensors for the last year in the last segment
  if set(active_sensors_per_year[year]) == set(active_sensors_per_year[segments[-1][1]]):
    segments[-1] = (segments[-1][0], year) # upate the ending year
  else:
    segments.append((year, year))

print(len(segments))

11


In [ ]:
for segment in segments:
  subset = df[(df['Year'] >= segment[0]) & (df['Year'] <= segment[1])] # segment[0] <= df['Year'] <= segment[1]
  grouped_subset = subset.pivot(index='New_Sensor_Name', columns='Date_Time', values='Hourly_Counts')
  save_data(subset, save_dir, save_subdirs.final_group, f"data_{segment[0]}_{segment[1]}.xlsx", index=True, rewrite=rewrite)
  save_data(grouped_subset, save_dir, save_subdirs.final_group, f"grouped_data_{segment[0]}_{segment[1]}.csv", index=True, rewrite=rewrite)

/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/4. final_group/data_2009_2013.xlsx updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/4. final_group/grouped_data_2009_2013.csv updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/4. final_group/data_2014_2014.xlsx updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/4. final_group/grouped_data_2014_2014.csv updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/4. final_group/data_2015_2015.xlsx updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/4. final_group/grouped_data_2015_2015.csv updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/4. final_group/data_2016_2016.xlsx updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/4. final_group/grouped_data_2016_2016.csv updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/4. final_group/data_2017_2017.xlsx updated.
/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls/4. fin

# <a id='toc9_'></a>[Process features (TBD)](#toc0_)

In [32]:
os.makedirs(save_dir / save_subdirs.add_fea, exist_ok=True)

sensor = pd.read_excel(save_dir / save_subdirs.merged / 'sensor_locations_processed.xlsx')

In [37]:
def fetch_amenities_count(lat, lon, distance, amenity_types):
  amenity_count = {}
  
  for amenity in amenity_types:
    # print(f"Fetching {amenity}...")
    try:
      gdf = ox.features_from_point((lat, lon), tags={'amenity': amenity}, dist=distance)
      if not gdf.empty:
        amenity_count[amenity] = len(gdf)
    except:
      pass
      
  return amenity_count

In [14]:
def sensor_amenities_count(sensor_df, distance, amenity_types):
  amenities_counts = []
  
  for index, row in sensor_df.iterrows():
    lat, lon = row['Latitude'], row['Longitude']
    count = fetch_amenities_count(lat, lon, distance, amenity_types)
    amenities_counts.append(count)
      
  amenities_count_df = pd.DataFrame(amenities_counts, index=sensor_df.index)
  return amenities_count_df

In [35]:
place_name = "Melbourne, Victoria, Australia"

# Fetch all geometries in the area
gdf = ox.geometries_from_place(place_name, tags={'amenity': True})

# Filter to keep only the 'amenity' tag
amenities = gdf[gdf['amenity'].notnull()]

amenity_types = amenities['amenity'].unique().tolist()

save_amenities = save_dir / save_subdirs.add_fea / 'amenity_types_melbourne.txt'
if not save_amenities.exists():
  with open(save_amenities, 'w') as f:
    for amenity in amenity_types:
      f.write(f"{amenity}\n")

  print("Amenity types saved to amenity_types_melbourne.txt")
else:
  amenity_types = pd.read_csv(save_amenities, header=None)[0].tolist()

/var/folders/mq/pnk708vd76xfjjjmy_4yn2lw0000gn/T/ipykernel_67868/1044787963.py:4: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  gdf = ox.geometries_from_place(place_name, tags={'amenity': True})


Amenity types saved to amenity_types_melbourne.txt


we have 212 amenities in the city

In [38]:
# amenity_types = [
#   'arts_centre', 'atm', 'bank', 'bar', 'bbq', 'bench', 'bicycle_parking', 'bicycle_rental', 'bus_station',
#   'cafe', 'car_rental', 'car_wash', 'cinema', 'clinic', 'college', 'community_centre', 'dentist', 'doctor',
#   'drinking_water', 'fast_food', 'ferry_terminal', 'fire_station', 'fuel', 'grave_yard', 'gym', 
#   'hospital', 'ice_cream', 'kindergarten', 'library', 'marketplace', 'monastery', 'nightclub', 'nursing_home',
#   'parking', 'pharmacy', 'place_of_worship', 'police', 'post_box', 'post_office', 'pub', 'public_building',
#   'recycling', 'restaurant', 'school', 'shelter', 'shop', 'taxi', 'telephone', 'theatre', 'toilets', 
#   'townhall', 'university', 'vending_machine', 'veterinary', 'waste_basket', 'waste_disposal', 'train_station'
# ]

# amenity_types = ['restaurant', 'cafe']

distance = 1000  # 1000 meters radius

amenities_count_1000 = sensor_amenities_count(sensor, distance, amenity_types)
amenities_count_1000

In [ ]:
amenities_count_1000['Sensor_Name'] = sensor['New_Sensor_Name']
save_data(amenities_count_1000, save_dir, save_subdirs.add_fea, 'amenities_count_1000.csv')

In [ ]:
distance = 500

amenities_count_500 = sensor_amenities_count(sensor, distance, amenity_types)
amenities_count_500

In [ ]:
amenities_count_500['Sensor_Name'] = sensor['New_Sensor_Name']
save_data(amenities_count_500, save_dir, save_subdirs.add_fea, 'amenities_count_500.csv')